In [1]:
import logging
from monai.data import DataLoader
import monai
from config.constants import (COVID_CASES_PATH, INFECTION_MASKS_PATH)
import torch
import numpy as np
import matplotlib.pyplot as plt
from monai.visualize import plot_2d_or_3d_image, matshow3d, blend_images
from pathlib import Path
from utils.helpers import load_images_from_path

2024-07-11 18:08:14.308479: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-11 18:08:15.149198: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
SPATIAL_SIZE = (32, 32, 32)
NUM_RAND_PATCHES = 4
LEVEL = -650
WIDTH = 1500
LOWER_BOUND_WINDOW_HRCT = LEVEL - (WIDTH // 2) 
UPPER_BOUND_WINDOW_HRCT = LEVEL + (WIDTH // 2)
LOWER_BOUND_WINDOW_CBCT = 0
UPPER_BOUND_WINDOW_CBCT = 255
SEED = 33


print(LOWER_BOUND_WINDOW_HRCT)
print(UPPER_BOUND_WINDOW_HRCT)
def get_hrct_transforms():
    return monai.transforms.Compose(
        [
            monai.transforms.LoadImaged(keys=('img', 'mask'), image_only=True, ensure_channel_first=True),
            monai.transforms.Orientationd(keys=('img', 'mask'), axcodes="PLI"),
            monai.transforms.ThresholdIntensityd(keys=("img",), threshold=LOWER_BOUND_WINDOW_HRCT, above=False, cval=LOWER_BOUND_WINDOW_HRCT),
            monai.transforms.ThresholdIntensityd(keys=("img",), threshold=UPPER_BOUND_WINDOW_HRCT, above=True, cval=UPPER_BOUND_WINDOW_HRCT),
            monai.transforms.ScaleIntensityRanged(keys=('img',), a_min=LOWER_BOUND_WINDOW_HRCT,
                                                  a_max=UPPER_BOUND_WINDOW_HRCT, b_min=0.0, b_max=1.0, clip=True, allow_missing_keys=True),
            monai.transforms.RandCropByPosNegLabeld(keys=('img', 'mask'), label_key="mask",
                                                    spatial_size=SPATIAL_SIZE, pos=1, neg=1,
                                                    num_samples=NUM_RAND_PATCHES, allow_smaller=True),
            monai.transforms.ToTensord(keys=("img", "mask")),
        ]
    )


def get_cbct_transforms():
    return monai.transforms.Compose(
        [
            monai.transforms.LoadImaged(keys=('img', 'mask'), image_only=True, ensure_channel_first=True),
            monai.transforms.Orientationd(keys=('img', 'mask'), axcodes="ALI"),
            monai.transforms.RandCropByPosNegLabeld(keys=('img', 'mask'), label_key="mask",
                                                    spatial_size=SPATIAL_SIZE, pos=1, neg=1,
                                                    num_samples=NUM_RAND_PATCHES, allow_smaller=True),
            monai.transforms.ScaleIntensityd(keys='img', minv=0.0, maxv=1.0),
            monai.transforms.ToTensord(keys=("img", "mask")),
        ]
    )

def get_val_hrct_transforms():
    return monai.transforms.Compose(
        [
            monai.transforms.LoadImaged(keys=('img', 'mask'), image_only=True, ensure_channel_first=True),
            monai.transforms.Orientationd(keys=('img', 'mask'), axcodes="PLI"),
            monai.transforms.ThresholdIntensityd(keys=("img",), threshold=LOWER_BOUND_WINDOW_HRCT, above=True, cval=LOWER_BOUND_WINDOW_HRCT),
            monai.transforms.ThresholdIntensityd(keys=("img",), threshold=UPPER_BOUND_WINDOW_HRCT, above=False, cval=UPPER_BOUND_WINDOW_HRCT),
            monai.transforms.ScaleIntensityd(keys='img', minv=0.0, maxv=1.0),
            monai.transforms.ToTensord(keys=("img", "mask")),
        ]
    )


def get_val_cbct_transforms():
    print("asdfas")
    return monai.transforms.Compose(
        [
            monai.transforms.LoadImaged(keys=('img', 'mask'), image_only=True, ensure_channel_first=True),
            monai.transforms.Orientationd(keys=('img', 'mask'), axcodes="ALI"),
            monai.transforms.ScaleIntensityd(keys='img', minv=0.0, maxv=1.0),
            monai.transforms.ToTensord(keys=("img", "mask")),
        ]
    )

def get_raw_transforms():
    return monai.transforms.Compose(
        [
            monai.transforms.LoadImaged(keys=('img', 'mask'), image_only=False, ensure_channel_first=True),
        ]
    )

def load_radiopaedia_from_path(path: str) -> list[str]:
    return sorted([str(f) for f in Path(path).iterdir() if f.suffix == '.gz' and f.is_file() and 'radiopaedia' in f.stem])

class CovidDataset(torch.utils.data.Dataset):
    def __init__(self, volumes, hrct_transform=None, cbct_transform=None):
        self.volumes = volumes
        self.hrct_transform = hrct_transform
        self.cbct_transform = cbct_transform

    def __len__(self):
        return len(self.volumes)

    def __getitem__(self, index):
        volume = self.volumes[index]

        if "coronacases" in volume["img"]:
            return self.hrct_transform(volume)
        else:
            return self.cbct_transform(volume)
        
# Load images and masks
logging.info(f"Loading images from {COVID_CASES_PATH}")
images = load_radiopaedia_from_path("../" + COVID_CASES_PATH)
labels = load_radiopaedia_from_path("../" + INFECTION_MASKS_PATH)


# Convert images and masks to a list of dictionaries with keys "img" and "mask"
data_dicts = np.array([{"img": img, "mask": mask} for img, mask in zip(images, labels)])
logging.debug(data_dicts)

shuffler = np.random.RandomState(SEED)
shuffler.shuffle(data_dicts)
data_dicts = list(data_dicts)



-1400
100


In [9]:
# Create a radiopaedia dataset
radiopaedia_dataset = CovidDataset(volumes=data_dicts, hrct_transform=get_raw_transforms(), cbct_transform=get_raw_transforms())
radiopaedia_loader = DataLoader(radiopaedia_dataset, batch_size=1, num_workers=2)

In [4]:
for i, data in enumerate(radiopaedia_loader):
    print(data["img"].min(), data["img"].max())

metatensor(-1021.) metatensor(1845.)
metatensor(-1021.) metatensor(2020.)
metatensor(-1023.) metatensor(2217.)
metatensor(-1021.) metatensor(1920.)


KeyboardInterrupt: 

In [92]:
print(LOWER_BOUND_WINDOW_HRCT)
print(UPPER_BOUND_WINDOW_HRCT)

-1400
100


In [41]:
def t():
    return monai.transforms.Compose(
        [
            # monai.transforms.LoadImaged(keys=('img', 'mask'), image_only=True, ensure_channel_first=True),
            monai.transforms.Orientationd(keys=('img', 'mask'), axcodes="PLI"),
            monai.transforms.ScaleIntensityRanged(keys=('img',), a_min=LOWER_BOUND_WINDOW_HRCT,
                                            a_max=UPPER_BOUND_WINDOW_HRCT, b_min=0.0, b_max=1.0, clip=True),
            monai.transforms.ToTensord(keys=("img", "mask")),
        ]
    )
sample_data = {
    'img': np.random.randint(-2000, 2000, (2, 128, 128, 128)),  # Example volume
    'mask': np.random.randint(0, 2, (2, 128, 128, 128))         # Example mask
}

transforms_pipeline = t()
transformed_data = transforms_pipeline(sample_data)

# Print some statistics of the normalized volume to verify
print(f"Min value (img): {transformed_data['img'].min()}")
print(f"Max value (img): {transformed_data['img'].max()}")
print(f"Mean value (img): {transformed_data['img'].mean()}")

Min value (img): 0.0
Max value (img): 1.0
Mean value (img): 0.6622612476348877


In [10]:
# import nibabel and load an image from data_paths and explore the min and max value of each image
import nibabel as nib
import os
import numpy as np

min = [100000,1000000,100000]
for i in range(len(data_dicts)):
    f = nib.load(data_dicts[i]["img"])
    print(f.header["srow_y"][1])
    if min[0] > abs(f.header["srow_x"][0]):
        min[0] = abs(f.header["srow_x"][0])
    if min[1] > abs(f.header["srow_y"][1]):
        min[1] = abs(f.header["srow_y"][1])
    if min[2] > abs(f.header["srow_z"][2]):
        min[2] = abs(f.header["srow_z"][2])
print(min)

0.683594
0.683594
0.683594
0.683594
0.683594
0.683594
0.683594
0.683594
0.683594
0.683594
[0.683594, 0.683594, 1.0]


In [11]:
for i, data in enumerate(radiopaedia_loader):
    print(data["img_meta_dict"]["affine"])

tensor([[[  -0.6836,   -0.0000,    0.0000,  159.0000],
         [  -0.0000,    0.6836,    0.0000,  174.0000],
         [   0.0000,    0.0000,    6.0000, -228.0000],
         [   0.0000,    0.0000,    0.0000,    1.0000]]], dtype=torch.float64)
tensor([[[  -0.6836,   -0.0000,    0.0000,  159.0000],
         [  -0.0000,    0.6836,    0.0000,  174.0000],
         [   0.0000,    0.0000,    6.0000, -390.0000],
         [   0.0000,    0.0000,    0.0000,    1.0000]]], dtype=torch.float64)
tensor([[[  -0.6836,   -0.0000,    0.0000,  159.0000],
         [  -0.0000,    0.6836,    0.0000,  174.0000],
         [   0.0000,    0.0000,    6.0000, -246.0000],
         [   0.0000,    0.0000,    0.0000,    1.0000]]], dtype=torch.float64)
tensor([[[  -0.6836,   -0.0000,    0.0000,  159.0000],
         [  -0.0000,    0.6836,    0.0000,  174.0000],
         [   0.0000,    0.0000,    6.0000, -264.0000],
         [   0.0000,    0.0000,    0.0000,    1.0000]]], dtype=torch.float64)
tensor([[[  -0.6836,   -0.00